| ![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-smaller.PNG)  | [National Science Data Fabric](https://nationalsciencedatafabric.org/) [Jupyter notebook](https://jupyter.org/) <br> created by  [Valerio Pascucci](http://cedmav.com/) and  [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html) ADD MORE HERE |  
|---|:---:|

# Creating a videoclip from material science data stored on the cloud


In [2]:
import OpenVisus                 

In [3]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
from PyQt5 import QtCore
from PyQt5 import QtWidgets
import sip

# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

# in case you are in debug mode
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

# change working directory as needed
#os.chdir(r"D:\pascucci\ppt\2022\Panias-talk\views")
print("current working directory",os.getcwd())

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

# use this function to create a Viewer, solves the problem of window not raising
viewer=None
def CreateViewer():
    global viewer
    
    if viewer is not None:
        del viewer
    
    viewer=PyViewer()
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    viewer_py.setVisible(True)
    viewer_py.show()
    viewer_py.setFocus()
    viewer_py.showMaximized()
    viewer_py.activateWindow()
    print(time.time(),"Viewer created")
    return viewer

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets', 'PyQt5.QtSvg']
current working directory /home/xuanhuang/projects/vis_interface/jupter_notebook_example
QT_PLUGIN_PATH /home/xuanhuang/.local/lib/python3.10/site-packages/PyQt5/Qt/plugins


In [4]:
# do not change this cell for very good reasons (otherwise Qt5 loop won't work)
# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
import time
time.sleep(2)

Define some utilities to perform viewer animations

In [5]:
# //////////////////////////////////////////////////////////////
"""
RunActions is needed to schedule actions and execute them as soon as the viewer is idle
Example:

    run_actions.addAction(fn,arg=0)

will run later.
If you want to execute an immediate action:

    fn(arg)

"""
class RunActions:
    
    def __init__(self):
        self.v=[]
        self.cursor=0
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.onTimer)
  
    def addAction(self, fn,**kwargs):
        self.v.append(lambda : fn(**kwargs))
        
    def addSleep(self,msec):
        self.v.append(msec)
        
    def onTimer(self):
        
        # if the last action was an action and the viewer is still running, I need to wait
        if self.cursor>0 and callable(self.v[self.cursor-1]) and viewer.isRunning():
            return
        
        if self.cursor>= len(self.v):
            return
        
        cur=self.v[self.cursor]
        self.cursor+=1
        
        # is a delay?
        if isinstance(cur,int):
            print("*** Sleep",cur)
            self.timer.start(cur) 
            
        # is a real action, run it and later wait for completition
        elif callable(cur):
            cur()
            self.timer.start(20) 
            
        # there must be a problem
        else:
            raise Exception("internal error")
   
    def start(self): 
        print("RunAction started")
        self.timer.start(1) 
        
    def stop(self):
        print("RunAction stopped")
        self.timer.stop()
        
def SetTime(time=0):
    print("*** SetTime",time)
    action=StringTree.fromString(f"""<SetCurrentTime target_id="nodes/dataset_time" value=" """+str(time)+""" " />"""); 
    viewer.execute(action)
    viewer.refreshAll()
    viewer.postRedisplay()
        
def RotateScene(axis=(0,0,1),angle=10):
    print("*** RotateScene",axis,angle)
    glcamera=viewer.getGLCamera()
    viewer.getGLCamera().setRotation(glcamera.getRotation() * Quaternion(Point3d(axis),math.radians(angle)))
    viewer.refreshAll()
    viewer.postRedisplay()
    

def TakeSnapshot(filename="temp.png",):
    print("*** TakeSnapshotAction",filename)  
    # viewer.takeSnapshot(False,filename) BROKEN
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    screenshot = QtWidgets.QApplication.primaryScreen().grabWindow(viewer_py.winId() )
    screenshot.save(filename)
    
def OpenScene(filename=""):
    print("*** OpenScene",filename)
    viewer.open(filename)
    
def DropSelection():
    print("*** DropSelection")
    viewer.dropSelection()
    
def HideDatasetBounds(uuid="dataset"):
    print("*** HideDatasetBounds",uuid)
    # **** make sure that the dataset node has UUID `dataset` (you can save an xml and inspect it) ***
    dataset_node=DatasetNode.castFrom(viewer.findNodeByUUID(uuid))
    if dataset_node:
        dataset_node.setShowBounds(False)
    else:
        print("Failed to find a node with uuid",uuid)
      
def TranslateSlice(slice=None,axis=0,offset=0):
    viewer.setSlicePosition(slice, axis, offset)

def setSlicePosition(slice=None,axis=0,offset=0):
    viewer.setSlicePosition(slice, axis, offset)

#run_actions.addAction(TakeSnapshot,filename="example.png")
#TakeSnapshot(filename="example.png")
    
print(time.time(),"Utilities defined")

1697508657.5924861 Utilities defined


In [6]:
ImageFilenameCounter = 0
def getImageFilename():
    global ImageFilenameCounter
    filename = os.path.join(os.getcwd(),f"snapshotb.{ImageFilenameCounter:04d}.png")
    ImageFilenameCounter+=1
    return filename 

In [7]:
import OpenVisus as ov
DatasetURL = r"http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1&cached=1"
DatasetURL = r"http://atlantis.sci.utah.edu/mod_visus?dataset=borg&cached=1"
DatasetURL = r"http://atlantis.sci.utah.edu/mod_visus?dataset=fly_scan_time-s&cached=1"
#DatasetURL = r"http://atlantis.sci.utah.edu/mod_visus?dataset=fly_scan_time-r&cached=1"
#DatasetURL = r"Scene2-bob.xml"
Colormap ="NDVI_Rainbow"
#palette_node = PaletteNode()
#palette_node = PaletteNode("pink")
Colormap = "gist_heat"

Volume   = ov.LoadDataset(DatasetURL)
VolNames = Volume.getFields
VolDims  = Volume.getLogicBox()
print("Scalar fields names=",VolNames)

(X1,Y1,Z1),(X2,Y2,Z2) = VolDims
print("Dataset [x,y,z] dimensions=",(X2-X1,Y2-Y1,Z2-Z1))


Scalar fields names= <bound method PyDataset.getFields of <OpenVisus.dataset.PyDataset object at 0x7fbd1f137f10>>
Dataset [x,y,z] dimensions= (2000, 2000, 2160)


In [8]:
#Get Dataset timesteps
VolumeTimesteps=Volume.getTimesteps()[23:23+11]
scans=[
		{"id":"112509","pos":(530 ,1.0)},
		{"id":"112512","pos":(1400,6.5)},
		{"id":"112515","pos":(2100,7.0)},
		{"id":"112517","pos":(2400,11)},
		{"id":"112520","pos":(3100,14)},
		{"id":"112522","pos":(3400,15)},
		{"id":"112524","pos":(3600,17)},
		{"id":"112526","pos":(3800,18)},
		{"id":"112528","pos":(4100,19)},
		{"id":"112530","pos":(4400,21)},
		{"id":"112532","pos":(4700,22)},
	]

for i in VolumeTimesteps:
    print (i,end=" ")


23 24 25 26 27 28 29 30 31 32 33 

In [10]:
DatasetURL = r"Scene2-bob.xml"
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()
viewer.render_palette=True

print(viewer.open(DatasetURL))
viewer.removeNode("volume")
print(len(viewer.findQueryNodes()))

dataset_tim_node=viewer.findNodeByUUID("dataset_time")
#action=StringTree.fromString(f"""<SetLightingEnabled target_id="nodes/{Xname}_render" value="True" />"""); 
#<RenderArrayNode uuid="Xslice_render" name="RenderArray" visible="True" lighting_enabled="True" palette_enabled="False" use_view_direction="False" max_num_slices="0" magnify_filter="9729" minify_filter="9729" render_type="OpenGL" />

action=StringTree.fromString(f"""<SetCurrentTime target_id="nodes/dataset_time" value="  5.000000  " />"""); 
action=StringTree.fromString(f"""<SetCurrentTime target_id="nodes/dataset_time" value=" """+str(50.000000)+""" " />"""); 


viewer.execute(action)


viewer.render_palettes=True
# default for addXXX functions
parent=None
fieldname=""
access_id=0

dataset = viewer.findNodeByUUID("dataset")

Xname="Xslice"
xslice=viewer.findNodeByUUID(Xname); 
Yname="Yslice"
yslice=viewer.findNodeByUUID(Yname); 
Zname="zslice"
zslice=viewer.findNodeByUUID(Zname); 

# remove all existing palettes
# palettes=viewer.findPaletteNodes()
# print (palettes)
# for node in palettes: 
#     viewer.removeNode(node)
    
# createe new palette and 'apply' for all query nodes
# palette_node = PaletteNode(Colormap)

# palette_node.setName("my-palette")
# viewer.addNode(dataset, palette_node)

Xrender=viewer.findNodeByUUID(f"{Xname}_render")
# ; viewer.connectNodes(palette_node, Xrender)
Yrender=viewer.findNodeByUUID(f"{Yname}_render")
# ; viewer.connectNodes(palette_node, Yrender)
Zrender=viewer.findNodeByUUID(f"{Zname}_render")
# ; viewer.connectNodes(palette_node, Zrender)

# action=StringTree.fromString(f"""<SetLightingEnabled target_id="nodes/{Xname}_render" value="True" />"""); 
# viewer.execute(action)
# action=StringTree.fromString(f"""<SetLightingEnabled target_id="nodes/{Yname}_render" value="True" />"""); 
# viewer.execute(action)
# action=StringTree.fromString(f"""<SetLightingEnabled target_id="nodes/{Zname}_render" value="True" />"""); 
# viewer.execute(action)


1697508727.54003 Viewer created
True
5


libpng warning: iCCP: known incorrect sRGB profile
QWidget::insertAction: Attempt to insert null action


rvice:785 933126:140449775867456 GET connection 4 wait 10 running 66 download 290.4KB  -  4400 kb/sec) status STATUS_OK url http://atlantis.sci.utah.edu/mod_visus?action=rangequery&block=130%20134%20162%20166%20131%20135%20163%20167&compression=zip&dataset=fly%5Fscan%5Ftime%2Ds&field=data&time=50%2E000000
201151457 QueryNode:182 933126:140449759082048 PointQuery finished msec Slice 90 level 3 / 4 / 24 / 34 dims 77 123 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201151458 QueryNode:227 933126:140449759082048 Query finished in 293 msec
201151462 RenderArrayNode:295 933126:140454863241664 Uploaded texture 77 123 1 uint8 9.2KB
201151475 RenderArrayNode:278 933126:140454863241664 got array dims 77 102 1 dtype uint8 scheduling texture upload 77 102 1 uint8 7.7KB got_new_data True msec 0
201151522 RenderArrayNode:295 933126:140454863241664 Uploaded texture 77 102 1 uint8 7.7KB
201151525 RenderArrayNode:278 933126:140454863241664 got array 

In [11]:
# example of rotation of the scene of 360 degrees
angleStep = 3 
angleStart = 1
angleRange = 360
TranslationRange = X2-X1
sleepStep = 50
lastAngle = angleStart
DryRun = False
#DryRun = True

In [13]:
run_actions=RunActions()
run_actions.start()
#run_actions.addAction(RotateScene,axis=(1,0,0),angle=180)

RunAction started


In [20]:

#viewer.render_palettes=True
# default for addXXX functions
parent=None
fieldname=""
access_id=0

print(time.time(),"OpenScene")
#viewer.removeNode("volume")

# uncomment as needed
run_actions.addAction(DropSelection)
run_actions.addAction(HideDatasetBounds,uuid="dataset")

run_actions.addSleep(50)

1697508903.2552464 OpenScene
*** DropSelection
*** HideDatasetBounds dataset
*** Sleep 50


In [21]:
#Play over time

for j,I in enumerate(VolumeTimesteps):   
    print ("timestep ",I," = flyscan-",scans[j]["id"])
    run_actions.addAction(SetTime,time=I)        
    run_actions.addAction(TakeSnapshot,filename=getImageFilename())        
    # important to sleep, otherwise take snapshot will not work (I know it makes no sense)
    run_actions.addSleep(sleepStep)


timestep  23  = flyscan- 112509
timestep  24  = flyscan- 112512
timestep  25  = flyscan- 112515
timestep  26  = flyscan- 112517
timestep  27  = flyscan- 112520
timestep  28  = flyscan- 112522
timestep  29  = flyscan- 112524
timestep  30  = flyscan- 112526
timestep  31  = flyscan- 112528
timestep  32  = flyscan- 112530
timestep  33  = flyscan- 112532


In [22]:
#Rotate while moving slices

for I in range(angleStart, angleRange,angleStep):
    if DryRun:
        if ((I < angleRange - (3*angleStep)) and
            (I > angleStart + (3*angleStep))) :
            continue
    lastAngle = I
    run_actions.addAction(RotateScene,axis=(0,0,1),angle=angleStep)
    Translation = I*TranslationRange/angleRange
    run_actions.addAction(setSlicePosition, slice=xslice, axis=0, offset=Translation)
    run_actions.addAction(setSlicePosition, slice=yslice, axis=1, offset=Translation)
    run_actions.addAction(setSlicePosition, slice=zslice, axis=2, offset=Translation)
    run_actions.addAction(TakeSnapshot,filename=getImageFilename())    
    
    # important to sleep, otherwise take snapshot will not work (I know it makes no sense)
    run_actions.addSleep(sleepStep)

*** SetTime 23
140449759082048 PointQuery finished msec Slice 4 level 3 / 4 / 24 / 34 dims 77 122 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201442247 QueryNode:227 933126:140449759082048 Query finished in 14 msec
201442313 RenderArrayNode:278 933126:140454863241664 got array dims 77 102 1 dtype uint8 scheduling texture upload 77 102 1 uint8 7.7KB got_new_data True msec 0
201442313 RenderArrayNode:278 933126:140454863241664 got array dims 77 122 1 dtype uint8 scheduling texture upload 77 122 1 uint8 9.2KB got_new_data True msec 0
201442325 IsoContourNode:528 933126:140449238996544 Marching cube on first 125 63 68 ntriangles 140646 done in 12 msec
201442348 RenderArrayNode:295 933126:140454863241664 Uploaded texture 77 122 1 uint8 9.2KB
201442348 RenderArrayNode:295 933126:140454863241664 Uploaded texture 77 102 1 uint8 7.7KB
201510213 QueryNode:170 933126:140449230603840 PointQuery msec 0 level 0 / 4 / 16 / 34 npoints 32 32 1 ...
2

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda : fn(**kwargs))
  File "/tmp/ipykernel_933126/821349286.py", line 105, in setSlicePosition
    viewer.setSlicePosition(slice, axis, offset)
  File "/home/xuanhuang/.local/lib/python3.10/site-packages/OpenVisus/VisusGuiPy.py", line 2070, in setSlicePosition
    return _VisusGuiPy.Viewer_setSlicePosition(self, *args)
TypeError: Wrong number or type of arguments for overloaded function 'Viewer_setSlicePosition'.
  Possible C/C++ prototypes are:
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double,Visus::Position)
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double)

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda :

201513013 QueryNode:205 933126:140449792652864 BoxQuery executeBoxQuery 0 / 3 / 16 / 34 ...
201513014 QueryNode:170 933126:140449759082048 PointQuery msec 0 level 0 / 4 / 16 / 34 npoints 32 64 1 ...
201513016 QueryNode:219 933126:140449792652864 BoxQuery executeBoxQuery IsoContour 0 / 3 / 16 / 34 done in 2 msec dims 32 16 34 dtype uint8 mem 17.0KB access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201513016 QueryNode:205 933126:140449792652864 BoxQuery executeBoxQuery 1 / 3 / 19 / 34 ...
201513016 QueryNode:182 933126:140449230603840 PointQuery finished msec Slice 3 level 0 / 4 / 16 / 34 dims 32 32 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201513016 QueryNode:170 933126:140449230603840 PointQuery msec 0 level 1 / 4 / 19 / 34 npoints 64 64 1 ...
201513018 QueryNode:182 933126:140449759082048 PointQuery finished msec Slice 3 level 0 / 4 / 16 / 34 dims 32 64 1 dtype uint8 access yes url http://atlanti

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda : fn(**kwargs))
  File "/tmp/ipykernel_933126/821349286.py", line 105, in setSlicePosition
    viewer.setSlicePosition(slice, axis, offset)
  File "/home/xuanhuang/.local/lib/python3.10/site-packages/OpenVisus/VisusGuiPy.py", line 2070, in setSlicePosition
    return _VisusGuiPy.Viewer_setSlicePosition(self, *args)
TypeError: Wrong number or type of arguments for overloaded function 'Viewer_setSlicePosition'.
  Possible C/C++ prototypes are:
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double,Visus::Position)
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double)

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda :

2023-10-16 20:15:13.426605 PyScriptingNode Got in input (1, 123, 71) uint8 origin Slice 
2023-10-16 20:15:13.428857 PyScriptingNode Output is  (1, 123, 71) uint8 msec 2 
2023-10-16 20:15:13.429107 PyScriptingNode Got in input (1, 109, 71) uint8 origin Slice 
2023-10-16 20:15:13.429366 PyScriptingNode Output is  (1, 109, 71) uint8 msec 1 
2023-10-16 20:15:13.430448 PyScriptingNode Got in input (68, 63, 125) uint8 origin IsoContour 
2023-10-16 20:15:13.430708 PyScriptingNode Output is  (68, 63, 125) uint8 msec 0 
513149 RenderArrayNode:295 933126:140454863241664 Uploaded texture 74 123 1 uint8 8.9KB
201513149 RenderArrayNode:295 933126:140454863241664 Uploaded texture 74 106 1 uint8 7.7KB
201513363 QueryNode:170 933126:140449230603840 PointQuery msec 0 level 0 / 4 / 16 / 34 npoints 32 32 1 ...
201513363 QueryNode:205 933126:140449792652864 BoxQuery executeBoxQuery 0 / 3 / 16 / 34 ...
201513363 QueryNode:170 933126:140449759082048 PointQuery msec 0 level 0 / 4 / 16 / 34 npoints 32 64 1 ..

...

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda : fn(**kwargs))
  File "/tmp/ipykernel_933126/821349286.py", line 105, in setSlicePosition
    viewer.setSlicePosition(slice, axis, offset)
  File "/home/xuanhuang/.local/lib/python3.10/site-packages/OpenVisus/VisusGuiPy.py", line 2070, in setSlicePosition
    return _VisusGuiPy.Viewer_setSlicePosition(self, *args)
TypeError: Wrong number or type of arguments for overloaded function 'Viewer_setSlicePosition'.
  Possible C/C++ prototypes are:
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double,Visus::Position)
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double)

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda :

*** TakeSnapshotAction /home/xuanhuang/projects/vis_interface/jupter_notebook_example/snapshotb.0292.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
s.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201552016 QueryNode:205 933126:140449792652864 BoxQuery executeBoxQuery 1 / 3 / 19 / 34 ...
201552016 QueryNode:182 933126:140449759082048 PointQuery finished msec Slice 2 level 0 / 4 / 16 / 34 dims 32 64 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201552016 QueryNode:170 933126:140449759082048 PointQuery msec 0 level 1 / 4 / 19 / 34 npoints 64 122 1 ...
201552020 QueryNode:182 933126:140449230603840 PointQuery finished msec Slice 5 level 1 / 4 / 19 / 34 dims 64 64 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201552020 QueryNode:170 933126:140449230603840 PointQuery msec 0 level 2 / 4 / 22 / 34 npoints 83 93 1 ...
201552020 QueryNode:219 933126:140449792652864 BoxQuery executeBoxQuer

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda : fn(**kwargs))
  File "/tmp/ipykernel_933126/821349286.py", line 105, in setSlicePosition
    viewer.setSlicePosition(slice, axis, offset)
  File "/home/xuanhuang/.local/lib/python3.10/site-packages/OpenVisus/VisusGuiPy.py", line 2070, in setSlicePosition
    return _VisusGuiPy.Viewer_setSlicePosition(self, *args)
TypeError: Wrong number or type of arguments for overloaded function 'Viewer_setSlicePosition'.
  Possible C/C++ prototypes are:
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double,Visus::Position)
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double)

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda :

*** TakeSnapshotAction /home/xuanhuang/projects/vis_interface/jupter_notebook_example/snapshotb.0293.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
can%5Ftime%2Ds
201552366 QueryNode:205 933126:140449792652864 BoxQuery executeBoxQuery 1 / 3 / 19 / 34 ...
201552367 QueryNode:182 933126:140449230603840 PointQuery finished msec Slice 4 level 0 / 4 / 16 / 34 dims 32 32 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201552367 QueryNode:170 933126:140449230603840 PointQuery msec 0 level 1 / 4 / 19 / 34 npoints 64 64 1 ...
201552368 QueryNode:182 933126:140449759082048 PointQuery finished msec Slice 5 level 0 / 4 / 16 / 34 dims 32 64 1 dtype uint8 access yes url http://atlantis.sci.utah.edu/mod_visus?dataset=fly%5Fscan%5Ftime%2Ds
201552368 QueryNode:170 933126:140449759082048 PointQuery msec 0 level 1 / 4 / 19 / 34 npoints 64 122 1 ...
201552369 QueryNode:219 933126:140449792652864 BoxQuery executeBoxQuery IsoContour 1 / 3 / 19 / 34 done in 4 m

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda : fn(**kwargs))
  File "/tmp/ipykernel_933126/821349286.py", line 105, in setSlicePosition
    viewer.setSlicePosition(slice, axis, offset)
  File "/home/xuanhuang/.local/lib/python3.10/site-packages/OpenVisus/VisusGuiPy.py", line 2070, in setSlicePosition
    return _VisusGuiPy.Viewer_setSlicePosition(self, *args)
TypeError: Wrong number or type of arguments for overloaded function 'Viewer_setSlicePosition'.
  Possible C/C++ prototypes are:
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double,Visus::Position)
    Visus::Viewer::setSlicePosition(Visus::QueryNode *,int,double)

Traceback (most recent call last):
  File "/tmp/ipykernel_933126/821349286.py", line 47, in onTimer
    cur()
  File "/tmp/ipykernel_933126/821349286.py", line 23, in <lambda>
    self.v.append(lambda :

In [14]:
%%time
import cv2
import os

image_folder = "./"
video_name = 'fly_scan_segmentedb.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 8, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

CPU times: total: 3.92 s
Wall time: 3.94 s
*** DropSelection
2023-10-09 14:25:51.661978 PyScriptingNode Got in input (1, 256, 256) uint8 origin Slice 2023-10-09 14:25:51.663978 PyScriptingNode Got in input (1, 128, 256) uint8 origin Slice 
2023-10-09 14:25:51.663978 PyScriptingNode Output is  (1, 128, 256) uint8 msec 0 
2023-10-09 14:25:51.663978 PyScriptingNode Got in input (135, 63, 125) uint8 origin IsoContour 
2023-10-09 14:25:51.664478 PyScriptingNode Output is  (135, 63, 125) uint8 msec 0 

2023-10-09 14:25:51.664478 PyScriptingNode Output is  (1, 256, 256) uint8 msec 2 
2023-10-09 14:25:52.031043 PyScriptingNode Got in input (270, 126, 250) uint8 origin IsoContour 
2023-10-09 14:25:52.032543 PyScriptingNode Output is  (270, 126, 250) uint8 msec 2 
2023-10-09 14:25:52.092554 PyScriptingNode Got in input (1, 512, 353) uint8 origin Slice 
2023-10-09 14:25:52.092554 PyScriptingNode Output is  (1, 512, 353) uint8 msec 1 
2023-10-09 14:25:52.149564 PyScriptingNode Got in input (1, 256

2023-10-09 14:26:21.836254 PyScriptingNode Got in input (1, 512, 353) uint8 origin Slice 
2023-10-09 14:26:21.837754 PyScriptingNode Output is  (1, 512, 353) uint8 msec 2 
2023-10-09 14:26:21.838254 PyScriptingNode Got in input (270, 126, 250) uint8 origin IsoContour 
2023-10-09 14:26:21.838254 PyScriptingNode Output is  (270, 126, 250) uint8 msec 0 
2023-10-09 14:26:21.898765 PyScriptingNode Got in input (1, 256, 352) uint8 origin Slice 
2023-10-09 14:26:21.898765 PyScriptingNode Output is  (1, 256, 352) uint8 msec 0 
2023-10-09 14:26:23.155485 PyScriptingNode Got in input (270, 252, 500) uint8 origin IsoContour 
2023-10-09 14:26:23.157986 PyScriptingNode Output is  (270, 252, 500) uint8 msec 3 
2023-10-09 14:26:23.402028 PyScriptingNode Got in input (1, 467, 352) uint8 origin Slice 
2023-10-09 14:26:23.404028 PyScriptingNode Output is  (1, 467, 352) uint8 msec 2 
2023-10-09 14:26:23.711583 PyScriptingNode Got in input (1, 562, 353) uint8 origin Slice 
2023-10-09 14:26:23.714084 PyScr

2023-10-09 14:26:52.215567 PyScriptingNode Got in input (270, 252, 500) uint8 origin IsoContour 
2023-10-09 14:26:52.219567 PyScriptingNode Output is  (270, 252, 500) uint8 msec 4 
2023-10-09 14:26:52.779665 PyScriptingNode Got in input (1, 562, 353) uint8 origin Slice 
2023-10-09 14:26:52.781165 PyScriptingNode Output is  (1, 562, 353) uint8 msec 2 
2023-10-09 14:26:52.903686 PyScriptingNode Got in input (1, 467, 352) uint8 origin Slice 
2023-10-09 14:26:52.903686 PyScriptingNode Output is  (1, 467, 352) uint8 msec 1 
2023-10-09 14:26:58.717703 PyScriptingNode Got in input (1, 562, 353) uint8 origin Slice 
2023-10-09 14:26:58.719703 PyScriptingNode Output is  (1, 562, 353) uint8 msec 1 
*** Sleep 50
*** SetTime 29
2023-10-09 14:26:59.214790 PyScriptingNode Got in input (1, 128, 128) uint8 origin Slice 
2023-10-09 14:26:59.216791 PyScriptingNode Output is  (1, 128, 128) uint8 msec 2 
2023-10-09 14:26:59.217290 PyScriptingNode Got in input (34, 16, 32) uint8 origin IsoContour 
2023-10-0

2023-10-09 14:27:26.220012 PyScriptingNode Got in input (1, 562, 353) uint8 origin Slice 
2023-10-09 14:27:26.222012 PyScriptingNode Output is  (1, 562, 353) uint8 msec 2 
*** TakeSnapshotAction D:\_PANIA_NEW_DATA\Video_pania_bob\snapshotb.0008.png
*** Sleep 50
*** SetTime 32
2023-10-09 14:27:26.594577 PyScriptingNode Got in input (34, 16, 32) uint8 origin IsoContour 
2023-10-09 14:27:26.594577 PyScriptingNode Output is  (34, 16, 32) uint8 msec 0 
2023-10-09 14:27:26.756106 PyScriptingNode Got in input (1, 128, 128) uint8 origin Slice 
2023-10-09 14:27:26.757606 PyScriptingNode Output is  (1, 128, 128) uint8 msec 2 
2023-10-09 14:27:26.758606 PyScriptingNode Got in input (1, 64, 128) uint8 origin Slice 
2023-10-09 14:27:26.758606 PyScriptingNode Output is  (1, 64, 128) uint8 msec 0 
2023-10-09 14:27:26.759106 PyScriptingNode Got in input (68, 32, 63) uint8 origin IsoContour 
2023-10-09 14:27:26.759106 PyScriptingNode Output is  (68, 32, 63) uint8 msec 1 
2023-10-09 14:27:26.981145 PySc

2023-10-09 14:27:46.776607 PyScriptingNode Got in input (1, 503, 328) uint8 origin Slice 
2023-10-09 14:27:46.779607 PyScriptingNode Output is  (1, 503, 328) uint8 msec 2 
2023-10-09 14:27:46.995645 PyScriptingNode Got in input (1, 566, 329) uint8 origin Slice 
2023-10-09 14:27:46.998145 PyScriptingNode Output is  (1, 566, 329) uint8 msec 2 
2023-10-09 14:27:48.591423 PyScriptingNode Got in input (1, 566, 329) uint8 origin Slice 
2023-10-09 14:27:48.593424 PyScriptingNode Output is  (1, 566, 329) uint8 msec 2 
2023-10-09 14:27:49.342556 PyScriptingNode Got in input (540, 252, 1000) uint8 origin IsoContour 
2023-10-09 14:27:49.344056 PyScriptingNode Output is  (540, 252, 1000) uint8 msec 2 
2023-10-09 14:27:51.410916 PyScriptingNode Got in input (1, 503, 328) uint8 origin Slice 
2023-10-09 14:27:51.412917 PyScriptingNode Output is  (1, 503, 328) uint8 msec 2 
*** TakeSnapshotAction D:\_PANIA_NEW_DATA\Video_pania_bob\snapshotb.0012.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
2023-10-09 

...

2023-10-09 14:32:20.581470 PyScriptingNode Got in input (1, 522, 512) uint8 origin Slice 
2023-10-09 14:32:20.583972 PyScriptingNode Output is  (1, 522, 512) uint8 msec 3 
2023-10-09 14:32:20.590974 PyScriptingNode Got in input (1, 231, 587) uint8 origin Slice 
2023-10-09 14:32:20.591472 PyScriptingNode Output is  (1, 231, 587) uint8 msec 0 
2023-10-09 14:32:20.963537 PyScriptingNode Got in input (1, 522, 547) uint8 origin Slice 
2023-10-09 14:32:20.966038 PyScriptingNode Output is  (1, 522, 547) uint8 msec 2 
*** TakeSnapshotAction D:\_PANIA_NEW_DATA\Video_pania_bob\snapshotb.0110.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
2023-10-09 14:32:22.265265 PyScriptingNode Got in input (1, 256, 128) uint8 origin Slice 
2023-10-09 14:32:22.265265 PyScriptingNode Output is  (1, 256, 128) uint8 msec 0 
2023-10-09 14:32:22.265765 PyScriptingNode Got in input (270, 126, 250) uint8 origin IsoContour 
2023-10-09 14:32:22.266265 PyScriptingNode Output is  (270, 126, 250) uint8 msec 0 
2023-10-09 14

2023-10-09 14:32:30.528711 PyScriptingNode Got in input (1, 525, 543) uint8 origin Slice 
2023-10-09 14:32:30.531211 PyScriptingNode Output is  (1, 525, 543) uint8 msec 2 
*** TakeSnapshotAction D:\_PANIA_NEW_DATA\Video_pania_bob\snapshotb.0115.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
2023-10-09 14:32:31.849941 PyScriptingNode Got in input (1, 256, 128) uint8 origin Slice 
2023-10-09 14:32:31.850441 PyScriptingNode Output is  (1, 256, 128) uint8 msec 0 
2023-10-09 14:32:31.850441 PyScriptingNode Got in input (270, 126, 250) uint8 origin IsoContour 
2023-10-09 14:32:31.850441 PyScriptingNode Output is  (270, 126, 250) uint8 msec 0 
2023-10-09 14:32:31.850941 PyScriptingNode Got in input (1, 254, 256) uint8 origin Slice 
2023-10-09 14:32:31.850941 PyScriptingNode Output is  (1, 254, 256) uint8 msec 0 
2023-10-09 14:32:31.926954 PyScriptingNode Got in input (1, 512, 256) uint8 origin Slice 
2023-10-09 14:32:31.926954 PyScriptingNode Output is  (1, 512, 256) uint8 msec 0 
2023-10-09 14

2023-10-09 14:32:40.218404 PyScriptingNode Got in input (1, 512, 505) uint8 origin Slice 
2023-10-09 14:32:40.220904 PyScriptingNode Output is  (1, 512, 505) uint8 msec 2 
2023-10-09 14:32:40.220904 PyScriptingNode Got in input (1, 305, 512) uint8 origin Slice 
2023-10-09 14:32:40.220904 PyScriptingNode Output is  (1, 305, 512) uint8 msec 0 
2023-10-09 14:32:40.364430 PyScriptingNode Got in input (540, 252, 500) uint8 origin IsoContour 
2023-10-09 14:32:40.364430 PyScriptingNode Output is  (540, 252, 500) uint8 msec 0 
2023-10-09 14:32:40.467448 PyScriptingNode Got in input (1, 539, 505) uint8 origin Slice 
2023-10-09 14:32:40.469948 PyScriptingNode Output is  (1, 539, 505) uint8 msec 3 
2023-10-09 14:32:40.470448 PyScriptingNode Got in input (1, 305, 530) uint8 origin Slice 
2023-10-09 14:32:40.470448 PyScriptingNode Output is  (1, 305, 530) uint8 msec 0 
*** TakeSnapshotAction D:\_PANIA_NEW_DATA\Video_pania_bob\snapshotb.0120.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
2023-10-09 14

2023-10-09 14:32:48.027770 PyScriptingNode Got in input (270, 252, 500) uint8 origin IsoContour 
2023-10-09 14:32:48.029270 PyScriptingNode Output is  (270, 252, 500) uint8 msec 2 
2023-10-09 14:32:48.093782 PyScriptingNode Got in input (1, 512, 438) uint8 origin Slice 
2023-10-09 14:32:48.094282 PyScriptingNode Output is  (1, 512, 438) uint8 msec 0 
2023-10-09 14:32:48.094782 PyScriptingNode Got in input (1, 366, 444) uint8 origin Slice 
2023-10-09 14:32:48.094782 PyScriptingNode Output is  (1, 366, 444) uint8 msec 1 
2023-10-09 14:32:48.343828 PyScriptingNode Got in input (1, 552, 438) uint8 origin Slice 
2023-10-09 14:32:48.346826 PyScriptingNode Output is  (1, 552, 438) uint8 msec 3 
2023-10-09 14:32:48.900923 PyScriptingNode Got in input (1, 552, 438) uint8 origin Slice 
2023-10-09 14:32:48.902423 PyScriptingNode Output is  (1, 552, 438) uint8 msec 1 
*** TakeSnapshotAction D:\_PANIA_NEW_DATA\Video_pania_bob\snapshotb.0125.png
*** Sleep 50
*** RotateScene (0, 0, 1) 3
2023-10-09 14

2023-10-09 14:32:55.289539 PyScriptingNode Got in input (1, 256, 352) uint8 origin Slice 
2023-10-09 14:32:55.293541 PyScriptingNode Output is  (1, 256, 352) uint8 msec 4 
2023-10-09 14:32:55.352051 PyScriptingNode Got in input (1, 512, 353) uint8 origin Slice 
2023-10-09 14:32:55.352051 PyScriptingNode Output is  (1, 512, 353) uint8 msec 0 
2023-10-09 14:32:55.352550 PyScriptingNode Got in input (270, 252, 500) uint8 origin IsoContour 
2023-10-09 14:32:55.352550 PyScriptingNode Output is  (270, 252, 500) uint8 msec 0 
2023-10-09 14:32:55.587591 PyScriptingNode Got in input (1, 562, 353) uint8 origin Slice 
2023-10-09 14:32:55.590091 PyScriptingNode Output is  (1, 562, 353) uint8 msec 2 
2023-10-09 14:32:55.590592 PyScriptingNode Got in input (1, 467, 352) uint8 origin Slice 
2023-10-09 14:32:55.590592 PyScriptingNode Output is  (1, 467, 352) uint8 msec 1 
2023-10-09 14:32:56.174194 PyScriptingNode Got in input (1, 562, 353) uint8 origin Slice 
2023-10-09 14:32:56.176694 PyScriptingNod